In [2]:
#import needed package
import asyncio
import httpx
import pandas as pd
import secret
import refresh
import requests
from time import perf_counter

In [3]:
#credentials
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [39]:
df = pd.read_excel('vincere_restore_file.xlsx')
df

,Vincere ID,Record ID,New First Name,New Last Name,Original - Email,New - Email
0,65359,19261278,Faeez Abdul,Razak,faeezrazak@gmail.com,faeezrazak@gmail.com
1,64524,19261768,Dražen,Janjiček,d.janjicek@gmail.com,d.janjicek@gmail.com
2,82599,19262852,Giorgio,Nava,nagio80@hotmail.com,nagio80@hotmail.com
3,90129,19262885,Fatima Joy,Jacinto-Loto,fatimajoyjacinto@gmail.com,fatimajoyjacinto@gmail.com
4,70506,19262888,Eng,Beng Tan,tanengbeng@gmail.com,tanengbeng@gmail.com
...,...,...,...,...,...,...
191,74855,19290430,Sing,Hwei Goh,anngohsh@gmail.com,anngohsh@gmail.com
192,69932,19290524,Željko,Spasojević,zex_ns@yahoo.com,zex_ns@yahoo.com
193,69376,19290671,Saw Ping,Yap,sawping93@gmail.com,sawping93@gmail.com
194,64489,19291174,Uroš,Ilić,ilic.z.uros@gmail.com,ilic.z.uros@gmail.com


In [4]:
#sem_GET = asyncio.Semaphore(100)
sem_PUT = asyncio.Semaphore(10)
    
#determining if it has the custom field data
def add_updated_data(data: dict, first_name: str, last_name: str, email: str) -> dict:
    data['first_name'] = first_name
    data['last_name'] = last_name
    data['email'] = email
    return data
    
#requesting data first
def request_data(id: int) -> dict:
    with httpx.Client() as client:
        r = client.get(f'https://bps-world.vincere.io/api/v2/candidate/{id}/', headers=headers)
        if r.status_code == 200:
            response = r.json()
            return response
        else:
            errorID_get[id] = r.status_code
            return {}

#update data
async def update_data(id: int, first_name: str, last_name: str, email: str) -> None:
    async with sem_PUT:
        async with httpx.AsyncClient() as client:
            response_data = request_data(id)
            data_container = add_updated_data(response_data, first_name, last_name, email)
            r = await client.put(f'https://bps-world.vincere.io/api/v2/candidate/{id}/', headers=headers, json=data_container)
            if r.status_code == 200:
                success[id] = r.status_code
                return
            else:
                errorID_put[id] = r.status_code
                return
                
#store errors for future reference
errorID_get = {}
errorID_put = {}
success = {}

async def main():
    vincere_id = 85738
    first_name = 'Hey Joshua'
    last_name = 'Welcome to API World'
    email = 'welcome@gmail.com'
    await update_data(vincere_id, first_name, last_name, email)
#    for a, b, c, d in zip(df['Vincere ID'], df['New First Name'], df['New Last Name'], df['New - Email']):
#        await update_data(a, b, c, d)

In [5]:
#trigger the whole function
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

time taken: 3.2433796999999913  seconds


In [43]:
len(success)

196

In [44]:
errorID_get

{}

In [45]:
errorID_put

{}